# Calibration Rectangle 

150mm x 50mm rectangle for calibration purposes
splay the image.

In [ ]:
import gcode
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# DOE Configuration
laserin_speed=200
laserin_power=200

In [ ]:
def init():
    prog = gcode.GCode()
    prog.G92(X=0, Y=0, Z=0) 
    prog.G21() # Metric
    prog.G90() # Absolute
    return prog

In [ ]:
def calibration_border(laserin_speed, laserin_power):
    axis_prog=gcode.GCode()
    axis_prog.G0(X=0, Y=0, Z=0, F=300)
    axis_prog.M4(S=laserin_power)
    axis_prog.G1(X=0, Y=0, Z=0, F=laserin_speed)
    axis_prog.G1(X=0, Y=160, Z=0, F=laserin_speed)
    axis_prog.G0(X=60, Y=0, Z=0)
    axis_prog.G1(X=0, Y=0, Z=0, F=laserin_speed)
    axis_prog.G1(X=10, Y=10, Z=0, F=laserin_speed)
    axis_prog.M5()
    return axis_prog

In [ ]:
init()+calibration_border(laserin_speed, laserin_power)

In [ ]:
doe=gcode.GCode()
X_pos = 10
Ys = np.array([10, 160])
    
for laserin_power in [75, 100, 125, 150, 175, 200, 255]:
    print(f"Power Test: {laserin_power}")
    for x_spacing in [0, 0.1, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2, 2.5]:
        X_pos+=x_spacing
        X_pos=np.round(X_pos, 4)
        #
        doe.G0(X=X_pos, Y=Ys[0])
        #
        doe.M4(S=laserin_power)
        doe.G1(X=X_pos, Y=Ys[0])
        doe.G1(X=X_pos, Y=Ys[1])
        doe.M5()
        #
        doe.G0(X=X_pos, Y=Ys[1])
        #
        print(f"\tX:{X_pos}\tY:{Ys[0]}:{Ys[1]}")
        Ys = np.flip(Ys)
    X_pos+=5
# 5 mm between DOE tests.
X_pos

In [ ]:
import requests

esphome_cfg = {
    "host": "cnc.lan",
    "id": "cnc_relay"
}

base_url = f"http://{esphome_cfg['host']}/switch/{esphome_cfg['id']}"
base_url

In [ ]:
r = requests.post(f"{base_url}/turn_on")
assert r.status_code == 200

r = requests.get(f"{base_url}")
assert r.json()["state"] == "ON"
assert r.json()["value"]

r = requests.post(f"{base_url}/turn_off")
assert r.status_code == 200


r = requests.get(f"{base_url}")
assert r.json()["state"] == "OFF"
assert not r.json()["value"]

In [ ]:
import sys

# https://github.com/jed-frey/build_opencv
sys.path.append("/opt/opencv4/lib/python3.8/site-packages")

import cv2
from PIL import Image
from IPython.display import display

In [ ]:
import timeout_decorator

class VideoCapture2(cv2.VideoCapture):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    @property
    def frame(self):
        ret, frame = self.read()
        if ret:
            return frame
        else:
            raise Exception("Reading Failed")
        
    def pil_img(self):
        img = cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB)
        return Image.fromarray(img)
    
    def __enter__(self):
        return self
    
    @timeout_decorator.timeout(10)
    def __exit__(self,type, value, traceback):
        while self.isOpened():
            self.release() 

            
import IPython.core.magic
@IPython.core.magic.register_line_magic
def webcam(*args, **kwargs):
    webcams = [arg for arg in args[0].split(" ")]
    for webcam in webcams:
        if webcam.startswith("/dev/video"):
            webcam = int(webcam.split("/dev/video")[-1])
        else:
            try:
                webcam = int(webcam)
            except:
                pass
        with VideoCapture2(webcam) as cap:
            img = cap.pil_img()
            display(img)

In [ ]:
import os
auth = os.environ["WEBCAM_AUTH"]
ip   = os.environ["WEBCAM_IP"]

In [ ]:
%webcam /dev/video0 rtsp://$auth@$ip/cam/realmonitor?channel=1&subtype=0

In [ ]:
r = requests.post(f"{base_url}/turn_on")
assert r.status_code == 200

r = requests.get(f"{base_url}")
assert r.json()["state"] == "ON"
assert r.json()["value"]

import grbl

cnc = grbl.Grbl(port="/dev/ttyUSB0")

cnc.home()

In [ ]:
cnc.serial.close()

In [ ]:
with grbl.Grbl(port="/dev/ttyUSB0") as cnc:
    cnc.home()

In [ ]:
init()+calibration_border(laserin_speed, laserin_power)

In [ ]:
doe

In [ ]:
full_program = init()+calibration_border(laserin_speed, laserin_power)+doe

In [ ]:
full_program.save("laser_spacing_power_doe.ngc")

In [ ]:
foo = gcode.GCode.load("laser_spacing_power_doe.ngc")
#TODO Make Class Function

In [ ]:
foo = gcode.GCode(machine=cnc)
foo.load("laser_spacing_power_doe.ngc")


In [ ]:
foo

In [ ]:
reloaded_program = gcode.GCode()
reloaded_program.load("laser_spacing_power_doe.ngc")
with grbl.Grbl(port="/dev/ttyUSB0") as cnc:
    cnc.home()
    cnc.run(reloaded_program)
    cnc.home()

In [ ]:
with grbl.Grbl(port="/dev/ttyUSB0") as cnc:
    cnc.reset()
    cnc.cmd("$X")
    cnc.cmd("G0X-25F300")
    cnc.cmd("$X")
    cnc.home()

In [ ]:
cnc = grbl.Grbl(port="/dev/ttyUSB0")

In [ ]:
cnc.status

In [ ]:
cnc.reset()

In [ ]:
cnc.cmd("$X")

In [ ]:
cnc.cmd("G91G21")

In [ ]:
cnc.cmd("G0X-10F100")

In [ ]:
def init():
    prog = gcode.GCode()
    prog.G92(X=0, Y=0, Z=0) 
    prog.G21() # Metric
    prog.G90() # Absolute
    return prog


X_pos = 15
laserin_speed=200
Ys = np.array([10, 60])

def calibration_border():
    laserin_power=255
    laserin_speed=100
    X_pos = 15
    Ys = np.array([10, 60])
    axis_prog=gcode.GCode()
    axis_prog.G0(X=0, Y=0, Z=0, F=300)
    axis_prog.M3(S=int(laserin_power))
    axis_prog.G1(X=0, Y=0, Z=0, F=laserin_speed)
    axis_prog.G1(X=0, Y=Ys[1], Z=0, F=laserin_speed)
    axis_prog.G0(X=X_pos+150, Y=0, Z=0)
    axis_prog.G1(X=0, Y=0, Z=0, F=laserin_speed)
    axis_prog.G1(X=X_pos, Y=Ys[0], Z=0, F=laserin_speed)
    axis_prog.M5()
    return axis_prog

doe=gcode.GCode()
for laserin_power in [75, 100, 125, 150, 175, 200, 255]:
    print(f"Power Test: {laserin_power}")
    for x_spacing in [0, 0.1, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2, 2.5]:
        X_pos+=x_spacing
        X_pos=np.round(X_pos, 4)
        #
        doe.G0(X=X_pos, Y=Ys[0])
        #
        doe.M3(S=int(laserin_power))
        doe.G1(X=X_pos, Y=Ys[0], F=laserin_speed)
        doe.G1(X=X_pos,Y=Ys[1], F=laserin_speed)
        doe.M5()
        #
        doe.G0(X=X_pos, Y=Ys[1])
        #
        print(f"\tX:{X_pos}\tY:{Ys[0]}:{Ys[1]}")
        Ys = np.flip(Ys)
    X_pos+=5
    

In [ ]:
doe

In [ ]:
with grbl.Grbl(port="/dev/ttyUSB0") as cnc:
    cnc.reset()
    cnc.home()
    cnc.run(init())
    cnc.run(calibration_border())
    cnc.run(doe)
    cnc.home()

In [ ]:
doe